### Get submissions (all documents) from saved pickle files

In [4]:
import pickle

In [5]:
with open('new_reddit_topics.pkl', 'rb') as pick:
    submissions = pickle.load(pick)

In [6]:
submissions.head()

,author,comms_num,created,id,score,text,url,date_created
0,Cengineeringquestion,3,1.527654e+09,8n2alt,1,Writing your own letter of Rec? So the profess...,https://www.reddit.com/r/gradadmissions/commen...,2018-05-30 00:14:49
1,Saturdalia,2,1.527652e+09,8n23o7,11,Getting academic references when you're long r...,https://www.reddit.com/r/gradadmissions/commen...,2018-05-29 23:51:51
2,HoldenCoughfield,1,1.527650e+09,8n1rna,1,Applying to PhD Programs vs. Masters - Initial...,https://www.reddit.com/r/gradadmissions/commen...,2018-05-29 23:11:21
3,Eldar333,1,1.527650e+09,8n1q2x,1,Contacting POIs Hey rising undergrad senior bi...,https://www.reddit.com/r/gradadmissions/commen...,2018-05-29 23:06:06
4,gohgoomah,1,1.527641e+09,8n0ld0,4,Seeking application advice to MS in Mental Hea...,https://www.reddit.com/r/gradadmissions/commen...,2018-05-29 20:45:47


In [7]:
submissions['date_created'].apply(lambda x: x.week)

0      22
1      22
2      22
3      22
4      22
5      22
6      22
7      22
8      22
9      22
10     22
11     22
12     22
13     22
14     22
15     22
16     22
17     22
18     22
19     22
20     22
21     22
22     22
23     22
24     22
25     22
26     22
27     22
28     22
29     22
       ..
958    10
959    10
960    10
961    10
962    10
963    10
964    10
965    10
966    10
967    10
968    10
969    10
970    10
971    10
972    10
973    10
974    10
975    10
976    10
977    10
978    10
979    10
980    10
981    10
982    10
983    10
984    10
985    10
986    10
987    10
Name: date_created, Length: 988, dtype: int64

### TF-IDF and NMF with final stopwords, mindf/maxdf, no. iterations, no. topics from tuning

In [73]:
import nltk
from nltk.corpus import stopwords

In [74]:
stop = stopwords.words('english')
stop += ['just', 'like', 'know', 'want', 'really', 'got', 'people', 'feel', 'would']
stop += ['would', 'could', 'should', 'us', 'we', 'great', 'good', 'might', 'may']
stop += ['get', 'go', 'anyone', 'one', 'take', 'too', 'also', 'going', 'getting']
stop = set(stop)

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

t_vect = TfidfVectorizer(stop_words=stop, ngram_range=(1,1), token_pattern="\\b[a-z][a-z]+\\b", min_df=50, max_df=.75)

t_vect.fit(submissions['text'])

t = t_vect.transform(submissions['text'])

n_topics = 15
nmf_new = NMF(
    n_components=n_topics,
    random_state=2,
    verbose=1,
    max_iter=1000
)

nmf_doc_top = nmf_new.fit_transform(t)

violation: 1.0
violation: 0.4134376708186488
violation: 0.284973088757323
violation: 0.20270107939183113
violation: 0.15548344525526775
violation: 0.13591764914202123
violation: 0.12745769861291184
violation: 0.11641158263211622
violation: 0.09910570119482268
violation: 0.08119716347696888
violation: 0.06911281741325283
violation: 0.06095610793838764
violation: 0.054344106380561696
violation: 0.04894801109233447
violation: 0.04390475942752227
violation: 0.039607980617836214
violation: 0.036216001313630786
violation: 0.03312392272784891
violation: 0.030611080691981343
violation: 0.02849654796791091
violation: 0.026529675892401475
violation: 0.024938493351212523
violation: 0.02358688281807608
violation: 0.022449711962378598
violation: 0.02140275171027888
violation: 0.02042982645589974
violation: 0.01951732411215377
violation: 0.018611575711585934
violation: 0.01805281821698337
violation: 0.017518777765592995
violation: 0.016903378951256123
violation: 0.01622383896887514
violation: 0.0154

### Get top words for each topic

In [87]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

print_top_words(nmf_new, t_vect.get_feature_names(), 10)

Topic #0: school, grad, graduate, schools, undergrad, degree, time, year, advice, job
Topic #1: email, back, heard, hear, still, waiting, weeks, applied, week, anything
Topic #2: cs, ms, science, computer, better, courses, math, degree, opportunities, job
Topic #3: research, experience, year, lab, work, time, summer, project, working, years
Topic #4: application, process, need, decision, already, fall, applying, applications, time, list
Topic #5: masters, programs, apply, degree, looking, applying, accepted, need, thing, international
Topic #6: university, engineering, universities, state, help, international, need, student, guys, students
Topic #7: admission, computer, science, decision, fall, recently, things, offer, received, couple
Topic #8: gpa, major, college, classes, math, grades, courses, years, semester, year
Topic #9: program, master, science, post, interested, wondering, sure, advice, field, focus
Topic #10: letters, recommendation, letter, professor, professors, ask, write

### Name topics (manually done)

In [88]:
topic_list = {0: 'Schools1',
              1: 'Admissions1',
              2: 'CS_Math',
              3: 'Research',
              4: 'Application',
              5: 'Programs1',
              6: 'Schools2',
              7: 'Admissions2',
              8: 'Grades',
              9: 'Programs2',
              10: 'Letter_of_Recommendation',
              11: 'GRE',
              12: 'Offer',
              13: 'Interview',
              14: 'Programs3'
             }

### Do some exploratory analysis using the document/topic matrix and save TF-IDF and NMF structures

In [89]:
import pandas as pd
tfidfnmf_topics = pd.DataFrame(nmf_doc_top)
tfidfnmf_tmp = tfidfnmf_topics

In [90]:
tfidfnmf_topics['TopicNum'] = tfidfnmf_topics.idxmax(axis=1)
tfidfnmf_topics['TopicWt'] = tfidfnmf_topics.iloc[:, :15].max(axis=1)
tfidfnmf_topics['PrimaryTopic'] = [topic_list[x] for x in tfidfnmf_topics['TopicNum']]

In [91]:
tfidfnmf_topics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,TopicNum,TopicWt,PrimaryTopic
0,0.013881,0.014267,0.000000,0.000000,0.002508,0.000000,0.000000,0.000000,0.000000,0.122650,0.174294,0.000000,0.000342,0.000000,0.002942,10,0.174294,Letter_of_Recommendation
1,0.032322,0.020127,0.002968,0.076648,0.019005,0.000000,0.047177,0.000000,0.062382,0.001818,0.097441,0.000000,0.000000,0.000000,0.022731,10,0.097441,Letter_of_Recommendation
2,0.000000,0.000000,0.000000,0.052531,0.219587,0.089078,0.000000,0.000000,0.000000,0.008920,0.001835,0.003573,0.000000,0.004526,0.082262,4,0.219587,Application
3,0.027157,0.000000,0.000000,0.130491,0.102644,0.009701,0.000000,0.000000,0.053494,0.078663,0.018131,0.003474,0.000000,0.021361,0.117501,3,0.130491,Research
4,0.000000,0.000000,0.046989,0.046676,0.043812,0.000000,0.008768,0.042788,0.109760,0.037966,0.064394,0.061888,0.000000,0.023810,0.004044,8,0.109760,Grades


In [92]:
docs_by_topic = tfidfnmf_topics.groupby('PrimaryTopic')

In [93]:
topics_top_doc = docs_by_topic.idxmax()[['TopicWt']]
topics_top_doc['DocText'] = [submissions.iloc[x]['text'] for x in topics_top_doc['TopicWt']]
topics_top_doc

,TopicWt,DocText
PrimaryTopic,,
Admissions1,417,Who else is waiting to hear back about their a...
Admissions2,948,"SAIS admission decisions? Released ""online."" D..."
Application,352,Vague application portal update
CS_Math,166,nyu poly ms cs vs uva ms ds Which one should I...
GRE,63,What’s a good GRE score? Hi everyone! My diagn...
Grades,462,Importance of Major GPA Hi I have a quick ques...
Interview,660,Could you share some interview tips for me? We...
Letter_of_Recommendation,506,Recommendation Letters - What's more important...
Offer,885,Accepted an MA offer! Has anyone else accepted...


In [94]:
topics_top_doc_wt = docs_by_topic.max()[['TopicWt']]
topics_top_doc_wt

,TopicWt
PrimaryTopic,
Admissions1,0.304437
Admissions2,0.493266
Application,0.409696
CS_Math,0.342371
GRE,0.345367
Grades,0.221594
Interview,0.483564
Letter_of_Recommendation,0.332741
Offer,0.346359


In [95]:
topics_doc_count = docs_by_topic.count()[['TopicNum']]
topics_doc_count

,TopicNum
PrimaryTopic,
Admissions1,88
Admissions2,29
Application,49
CS_Math,48
GRE,63
Grades,75
Interview,31
Letter_of_Recommendation,61
Offer,80


In [96]:
with open('nmf_doc_top_5_29.pkl', 'wb') as piccle:
    pickle.dump(nmf_doc_top, piccle)

In [97]:
with open('nmf_mod_5_29.pkl', 'wb') as piccle2:
    pickle.dump(nmf_new, piccle2)

In [98]:
with open('tfidf_vect_5_29.pkl', 'wb') as piccle3:
    pickle.dump(t_vect, piccle3)

In [99]:
with open('topic_list_5_29.pkl', 'wb') as piccle4:
    pickle.dump(topic_list, piccle4)

### TSNE and bokeh plot

In [100]:
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca', metric='cosine')
tsne_nmf = tsne_model.fit_transform(nmf_doc_top)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 988 samples in 0.000s...
[t-SNE] Computed neighbors for 988 samples in 0.036s...
[t-SNE] Computed conditional probabilities for sample 988 / 988
[t-SNE] Mean sigma: 0.213500
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.342270
[t-SNE] Error after 1000 iterations: 1.047111


In [101]:
import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import save, reset_output, figure, show
from bokeh.models import HoverTool, ColumnDataSource
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f"
])

In [102]:
subm_text = submissions['text']
topic_nm = list(topic_list.values())
topic_num = list(tfidfnmf_topics['TopicNum'])
topic_name = list(tfidfnmf_topics['PrimaryTopic'])
max_topic_wt = list(tfidfnmf_topics['TopicWt'])

In [107]:
reset_output()

In [117]:
num_example = len(nmf_doc_top)

source = ColumnDataSource(data=dict(
    x = list(tsne_nmf[:,0]),
    y = list(tsne_nmf[:,1]),
    color=colormap[topic_num][:num_example],
    max_wt = max_topic_wt,
    text = list(subm_text)))

hover = HoverTool(tooltips='''
    topic_wt: @max_wt <br>
    text: <div style="width:350px">@text'</div>
''')

p_nmf2 = figure(plot_width=1200, plot_height=800,
                     #title=title,
                     tools=["pan",'wheel_zoom','box_zoom','reset','previewsave',hover],
                     x_axis_type=None, y_axis_type=None, min_border=1)
p_nmf2.circle(x='x', y='y', color='color', size=8, source=source)

topic_coord = np.empty((nmf_doc_top.shape[1], 2)) * np.nan
for topic in topic_name:
    if np.isnan(topic_coord).any():
        doc_index = int(topics_top_doc.loc[topic]['TopicWt'])
        top_index = [k for k, v in topic_list.items() if v == topic]
        topic_coord[top_index] = tsne_nmf[doc_index]

for i in range(nmf_doc_top.shape[1]):
    p_nmf2.text(topic_coord[i, 0], topic_coord[i, 1], [topic_nm[i]])

In [118]:
show(p_nmf2)

/anaconda3/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):


In [119]:
save(p_nmf2, 'gradadmission.html')

/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:138: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
/anaconda3/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):


'/Users/douglaslee/gradadmission.html'